In [12]:
import json
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import shape
import pandas as pd

In [13]:
precincts=json.load(open('first-phase/Washington DC/WashingtonDC-precinct.geojson','r'))
# precincts=precincts['features']
# print(len(precincts))

polygons=[]
# precincts[0]
cx,cy=0,0
for precinct in precincts['features']:
  precinct['precinct']=precinct['properties']['precinct']
  polygons.append([shape(precinct['geometry']),precinct['properties']['precinct']])

long=0
lat=0
for p in polygons:
  cord=p[0].centroid.coords[0]
  long+=cord[0]
  lat+=cord[1]
long/=len(polygons)
lat/=len(polygons)
with open('./washington-city.json','w') as file:
  json.dump({
    "latitude": lat, 
    "longitude": long,
  },file)
# precincts['features']
# polygons[0][0].centroid.coords[0]

In [18]:
df=pd.read_csv('./first-phase/Washington DC/Washington DC Db.csv')
# prev_crime_data=pd.read_csv('./wa')
df=df[0:10000]
list1=[]
for polygon in polygons:
  c=0
  count={'precinct': polygon[1],'total':0}
  # count=0
  for index,row in df.iterrows():
    if polygon[0].contains(Point(row['Longitude'],row['Latitude'])):
      if row['Offence'] not in count:
        count[row['Offence']]=0
      count[row['Offence']]+=1
      count['total']+=1
  list1.append(count)
list1

[{'precinct': 401,
  'total': 175,
  'Theft': 128,
  'Robbery': 38,
  'Assault': 7,
  'Murder': 1,
  'Sex Crime': 1},
 {'precinct': 201, 'total': 62, 'Theft': 56, 'Robbery': 6},
 {'precinct': 406,
  'total': 146,
  'Theft': 107,
  'Robbery': 33,
  'Assault': 5,
  'Sex Crime': 1},
 {'precinct': 402,
  'total': 119,
  'Theft': 89,
  'Robbery': 24,
  'Murder': 2,
  'Assault': 3,
  'Sex Crime': 1},
 {'precinct': 202,
  'total': 146,
  'Robbery': 30,
  'Theft': 111,
  'Assault': 3,
  'Sex Crime': 2},
 {'precinct': 403,
  'total': 126,
  'Theft': 87,
  'Robbery': 31,
  'Murder': 2,
  'Assault': 6},
 {'precinct': 203,
  'total': 140,
  'Sex Crime': 4,
  'Theft': 113,
  'Robbery': 16,
  'Assault': 7},
 {'precinct': 405,
  'total': 132,
  'Theft': 95,
  'Robbery': 32,
  'Assault': 4,
  'Sex Crime': 1},
 {'precinct': 404,
  'total': 191,
  'Theft': 148,
  'Robbery': 35,
  'Assault': 7,
  'Murder': 1},
 {'precinct': 407,
  'total': 120,
  'Theft': 80,
  'Robbery': 36,
  'Assault': 3,
  'Murder': 

In [14]:
# list1
# crime_data=pd.DataFrame.from_dict(list1)
# print(prev_crime_data.head(1))
# print(crime_data.head(1))
# x=crime_data.head(1).add(prev_crime_data.head(1))
# x
df.to_csv('seattle-precinct-wise-dataset.csv')

NameError: name 'df' is not defined

In [5]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default='chrome'
crime_data=pd.read_csv('seattle-precinct-wise-dataset.csv')
data=json.load(open('./seattle.json'))
lat=data['latitude']
long=data['longitude']

In [6]:
fig=px.choropleth_mapbox(crime_data,locations='precinct',featureidkey="properties.precinct",geojson=precincts,color='total',color_continuous_scale='ylorrd')
# fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(mapbox_style="carto-positron", 
                  mapbox_zoom=9.5,
                  mapbox_center={"lat": lat, "lon": long},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  uirevision='constant')
fig.write_html('first_figure-3.html')